In [1]:
# Requirements:
# !pip install rtdl
# !pip install libzero==0.0.4

In [2]:
from typing import Any, Dict

import numpy as np
import rtdl
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import zero
import pandas as pd

In [3]:
device = torch.device('cpu')
# Docs: https://yura52.github.io/zero/0.0.4/reference/api/zero.improve_reproducibility.html
zero.improve_reproducibility(seed=123456)

123456

### Data

In [4]:
from sklearn.datasets import fetch_kddcup99 
data = fetch_kddcup99(subset='train')

from pprint import pprint
print(list(data.target_names))

['labels']


In [5]:
df = pd.DataFrame(data.data.tolist(), columns=data.feature_names)
oldNames = data.feature_names

In [6]:
df["target"] = data.target

In [7]:
output = df.target.values
labels = set(output)
print('The different type of output labels are:',labels)
print('='*125)
print('No. of different output labels are:', len(labels))

The different type of output labels are: {b'rootkit.', b'neptune.', b'satan.', b'smurf.', b'loadmodule.', b'ftp_write.', b'portsweep.', b'warezclient.', b'pod.', b'buffer_overflow.', b'nmap.', b'warezmaster.', b'phf.', b'land.', b'guess_passwd.', b'multihop.', b'normal.', b'back.', b'spy.', b'ipsweep.', b'imap.', b'teardrop.', b'perl.'}
No. of different output labels are: 23


In [8]:
def onhotencode(_df, _col):
    _values = set(_df[_col].values)
    for v in _values:
        _df[v] = _df[_col].apply(lambda x : float(x == v) )
    return _df

In [9]:
for c in df.columns:
    if not c in ['protocol_type','service','flag', 'target']:
        df = df.drop(c, axis=1)

In [10]:
df = onhotencode(df, 'protocol_type')
df = onhotencode(df, 'service')
df = onhotencode(df, 'flag')

In [11]:
df = df.drop("protocol_type", axis=1)
df = df.drop("service", axis=1)
df = df.drop("flag", axis=1)

In [12]:
df["target"] = df["target"].apply(lambda x: str(x))

In [13]:
# df.to_csv("data/kdd_processed.csv")